# Image classification
*Thomas Dooms*

In the previous chapter, we discussed weight-based interpretability vs activation-based interpretability. We also explored bilinear layers and interaction matrices, a central object toward interpreting them. This chapter will cover small image classifiers trained on MNIST. Our interpretable, bilinear architecture will give us the tools necessary to understand what these models are learning. Previously, this was famously difficult; the Anthropic team once made the following statement [[ref](https://transformer-circuits.pub/2024/jan-update#mnist-sparse)].

> Our failure to understand MNIST models might be seen as an example of a more general trend: mechanistic interpretability has had little success in reverse engineering MLP layers in small but real neural networks. ... perhaps small models are just pathological in some way and don't have "crisp abstractions"!

We will show that small MNIST models actually learn very sensible structures once we understand how to look for them. This chapter is structured in a trial-and-error fashion, reflecting to some degree our process in interpreting these models. That said, let's dive right in.

### Setting up

The model we are using is a slight extension of our previous architecture. We now have an embedding matrix, followed by the same MLP as before. Little has changed beyond the model's dimensions and the fact that we are training on MNIST.

In [5]:
!git clone https://github.com/LaurenceWroe/bilinear-decomposition.git
%cd bilinear-decomposition

import sys, os
sys.path.append(os.path.abspath("."))



Cloning into 'bilinear-decomposition'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 175 (delta 79), reused 105 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (175/175), 17.35 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/content/bilinear-decomposition/bilinear-decomposition
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 1.7 MB/s eta 0:00:00


In [7]:
!pip install jaxtyping kornia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.4 MB/s eta 0:00:00


In [8]:
#%load_ext autoreload
#%autoreload 2


import torch
import plotly.express as px

from image import Model, MNIST
from einops import einsum
from kornia.augmentation import RandomGaussianNoise
from image import plot_eigenspectrum, plot_explanation

# Change this to "cpu" if you don't have a GPU
device = "cpu"

# Instantiate the model with the default configuration
model = Model.from_config(epochs=20).to(device)

# Load the MNIST dataset (directly on to a device for efficiency)
train, test = MNIST(train=True, device=device), MNIST(train=False, device=device)
metrics = model.fit(train, test)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.98MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 159kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.50MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.54MB/s]
train/loss: 0.101, train/acc: 0.972, val/loss: 0.123, val/acc: 0.965: 100%|██████████| 20/20 [00:41<00:00,  2.07s/it]


### Attempt 1: naive approach
Let's simply re-use the code from the previous chapter, with a slight modification to account for the embedding matrix.

In [ ]:
# .w_l and .w_r would return the left matrices for each layer. We only have one layer, so we index at 0.
l, r = model.w_l[0], model.w_r[0]

# We compute the same as previously, now taking into account the additional embedding matrix
b = einsum(model.w_u, l, r, model.w_e, model.w_e, "cls out, out hid1, out hid2, hid1 in1, hid2 in2 -> cls in1 in2")
b = 0.5 * (b + b.mT)

px.imshow(b[0].cpu(), color_continuous_midpoint=0, color_continuous_scale="RdBu")

We get ... something. It's not random, but it's rather hard to determine what is happening. The reason is simple: viewing images as vectors loses spatial information. This is aggravated by considering the interactions between these vectors, resulting in doubly distorted information. Ideally, we'd somehow turn this interaction matrix into a sum of vectors, which all capture meaningful information about the interactions. That way, we can actually visualize them.

### Attempt 2: decomposition into components
There are many matrix decompositions to achieve this. Due to the symmetric nature of our interaction matrices, we can use an especially nice decomposition: the eigendecomposition. Informally, the eigendecomposition converts a matrix into a collection of eigenvectors and eigenvalues. The eigenvector represents some matrix component, and the eigenvalue describes its importance. This is especially nice since we can order eigenvectors by relevance and analyze however many we wish, knowing we are doing so optimally.

So, let's implement this approach. First, we convert the MLP part of the model into its interaction matrices. Then, we perform the eigendecomposition on this. There are a few noteworthy details to this.
- This function only considers the last two dimensions for the decomposition; it views all others as a batch. Hence, it computes the decomposition for each output in our case, which is what we want.
- There are some variants of the eigendecomposition in PyTorch; since we know our matrix is symmetric, this one is best.

Lastly, we project the eigenvectors to the 'input space' and visualize them. For now, we simply visualize the most important (positive) eigenvector, which is stored at the last index.

In [ ]:
# Split the bilinear layer into the left and right components
l, r = model.w_lr[0].unbind()

# Compute the third-order (bilinear) tensor
b = einsum(model.w_u, l, r, "cls out, out in1, out in2 -> cls in1 in2")

# Symmetrize the tensor
b = 0.5 * (b + b.mT)

# Perform the eigendecomposition
vals, vecs = torch.linalg.eigh(b)

# Project the eigenvectors back to the input space
vecs = einsum(vecs, model.w_e, "cls emb comp, emb inp -> cls comp inp")

# Take the class (cls) for digit 0 and the last component (comp), which indicates the most positive eigenvalue
px.imshow(vecs[0, -1].view(28, 28).cpu(), color_continuous_midpoint=0, color_continuous_scale="RdBu")

While we see some spatial structure, this is still not what we expect (or what the paper shows). As we will see, this is actually a feature, not a bug! Essentially, since we did not regularize our model during training, it overfits. Specifically, it's picking up on some outlying pixels that probably don't occur often and uses those to memorize the class, hence their importance. This is not really what we want; we want to understand what a 'generalizing' model is doing.

=== Attempt 3: adding training regularization
Our ``.fit()`` function accepts augmentations (a small function that is applied to all inputs). Common augmentation (or regularization) includes rotation and translation. Experimentally, we found that these don't always work for MNIST since the model can still memorize certain outlying patterns. We did find input noise (adding some noise to each pixel) to work quite well, so let's now retrain a model using that.


In [ ]:
model = Model.from_config(epochs=20).to(device)
metrics = model.fit(train, test, RandomGaussianNoise(std=0.4))

In [ ]:
vals, vecs = model.decompose()
px.imshow(vecs[0, -1].view(28, 28).cpu(), color_continuous_midpoint=0, color_continuous_scale="RdBu")

Finally, we get discernable digits. With this setup, we can recreate the plots from the paper (after removing some visual bloat). We can get slightly cleaner features by training longer or adding slightly more noise.

In Appendix B of the paper, we perform a broader study of how different types of regularization and augmentation impact learned features. As stated above, many common augmentations don't remove overfitting. Instead of single pixels, it overfits based on small regions (according to the augmentation).

In [ ]:
# The main visualization code
fig = px.imshow(vecs[1:6, -1].view(-1, 28, 28).cpu(), color_continuous_midpoint=0, color_continuous_scale="RdBu", facet_col=0)

# Hide all the abundant information to get a clean plot
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.update_layout(showlegend=False).update_coloraxes(showscale=False)
fig.for_each_annotation(lambda a: a.update(text=""))

### Multiple eigenvectors per digit.

We've now recreated the result from the paper, but what does it actually mean? Let's write down the actual decomposition: $\sum_i \lambda_i (\textbf{v}_i \textbf{x}_i)^2$ where $\lambda_i$ are the eigenvalues and $\textbf{v}_i$ are the eigenvectors. We call $(\textbf{v}_i \textbf{x})^2$ the activation (as this is analogous to a normal activation, just in a different 'feature' space).

Interpreting eigenvectors is not like interpreting a heatmap where blue is a positive contribution and red is a negative contribution. Rather, the colors represent a form of constructive and destructive interference:
- If the input matches *none* of the colors, the activation will be *low*.
- If the input matches *either* color, the activation will be *high*.
- If the input matches *both* colors, the activation will be *low*.

This is somewhat analogous to an XOR-gate. Also, notice that the activation is either low or high, never negative (due to the square). The sign of the eigenvalue fully determines whether a feature will contribute positively or negatively. Eigenvectors with positive eigenvalues generally represent specific parts of digits that we expect (and have seen until now). Negative eigenvectors are somewhat less intuitive; they generally represent specific parts that are most destructive towards classifying a given digit, which is often hard to understand. Consider the digit 4; if one were to add a vertical line between the top parts, it would most likely be a 9. Hence, it makes sense for the model to strongly negatively match this region.

Let's visualize what these eigenvalues look like.

In [ ]:
# Visualize the eigenvalues for all digits
px.line(vals.cpu().T, template="plotly_white")

We see that in all cases, there are a few important eigenvalues (about 10 on each side).
Plotting these important vectors isn't difficult, but let's use a fancy function that puts everything nicely side-by-side.

In [ ]:
plot_eigenspectrum(model, digit=5)

### Using eigenvectors as explanations.
The features we get are nice, but are they actually good at explaining why a model makes a certain prediction? We can check this by measuring the activations for a given input and then plotting the features that activated most strongly. We plot both the top and bottom activating eigenvector for the three digits that had the highest logits. We see that, generally, only very few eigenvectors contribute significantly.

In [ ]:
plot_explanation(model, train.x[0])

### Future directions.

The decomposition above is only one way to analyze the interaction matrices. A good decomposition satisfies two criteria: short and interpretable. We satisfy the second criterion by considering features per digit, which are generally easier to understand. However, it may miss shared features, specific strokes that occur in multiple digits, which the model leverages. In that sense, our decomposition could be shorter. This becomes especially important when scaling up. It's currently somewhat unclear what the best way is to achieve both criteria. One possibility is dictionary learning, not SAEs, but rather 'Ye Olde' Lasso Regression.

Properly scaling this method remains an open question. Our proposed method decomposes the model top-down (output to input). At each step, we compute eigenvectors and use those as output directions for the next layers. One could perform a sort of beam search to extract the most important global directions. However, this leverages no shared features in the model, which is suboptimal. Clustering could possibly help in this.